# Item 46: Use descriptors for reusable `@property` methods

In [1]:
class Homework:
    
    def __init__(self):
        self._grade = 0
        
    @property
    def grade(self):
        return self._grade
    
    @grade.setter
    def grade(self, value):
        if not (0 <= value <=100):
            raise ValueError('Grade must be between 0 and 100')
        self._grade = value

This class is easy to use:

In [2]:
galileo = Homework()
galileo.grade = 95

Say the exam has multiple subjects, each with a separate grade 

In [3]:
class Exam:
    
    def __init__(self):
        self._writing_grade = 0
        self._math_grade = 0
        
    @staticmethod
    def _check_grade(value):
        if not (0 <= value <= 100):
            raise ValueError('Grade must be between 0 and 100')
            
    # The problem is that now we have to write properties for each grade.
    @property
    def writing_grade(self):
        return self._writing_grade
    
    @writing_grade.setter
    def writing_grade(self, value):
        self._check_grade(value)
        self._writing_grade = value
    
    @property
    def math_grade(self):
        return self._math_grade
    
    @math_grade.setter
    def math_grade(self, value):
        self._check_grade(value)
        self._math_grade = value
    

This seems quite cumbersome and tedious to extend/maintain. A better way is using the **descriptor protocol**:

In [4]:
from weakref import WeakKeyDictionary

class Grade:
    """This can be a grade for any subject!"""
    
    def __init__(self):
        # Avoid leaking memory. Item is removed from 
        # dict when the key ceases to exist.
        # This is needed because all the instances of `Exam`
        # will have access to a single instance of `Grade`.
        # This dict makes sure that if an instance of `Exam`
        # is no longer in use, the key is removed. Hence, the
        # garbage collector will succeed at removing the `Exam` object.
        self._values = WeakKeyDictionary() 
    
    def __get__(self, instance, instance_type):
        """
        :param instance: in this example is the Exam instance.
        :param instance_type: in this example is the class `__main__.Exam`.
        """
        if instance is None:
            return self
        return self._values.get(instance, 0)
    
    def __set__(self, instance, value):
        if not (0 <= value <= 100):
            raise ValueError('Grade must be between 0 and 100')
        self._values[instance] = value
        
class Exam:
    math_grade = Grade()
    writing_grade = Grade()
    science_grade = Grade()



In [11]:
first_exam = Exam()
first_exam.writing_grade = 82

second_exam = Exam()
second_exam.writing_grade = 97

print(first_exam.writing_grade)
print(second_exam.writing_grade)

82
97


## Things to remember
* Reuse the behaviour and validation of @property methods by defining your own descriptor classes.
* Use `WeakKeyDictionary` to ensure that your descriptor calsses don't cause memory leaks.
* Don't get **bogged down** trying to undestand exactly how `__getattribute__` uses the descriptor protocol for getting and setting classes.